In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import html5lib

In [2]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [3]:
# Get current S&P table and set header columns
sp500 = data[0]
drop_columns = ['SEC filings', 'Headquarters Location', 'Founded', 'GICS Sector',
               'GICS Sub Industry']
sp500.drop(columns = drop_columns, inplace = True)
# Standardizing CIK codes
sp500.CIK = sp500.CIK.astype(str)
sp500['CIK'] = sp500['CIK'].str.zfill(10)
# Renaming columns
sp500.rename(columns = {'Symbol':'TICKER', 'Security':'NAME', 'Date first added':'DATE'}, inplace = True)
# Adding variable column for added and removed tickers
sp500['VARIABLE'] = 'added_ticker'
# Filling in null values for dates
sp500.loc[sp500['DATE'].isnull(), 'DATE'] = '1957-01-01'
# Correcting date format for AT&T
sp500.loc[sp500[sp500['TICKER'] == 'T'].index, 'DATE'] = '1983-11-30'
# Convert to datetime
sp500['DATE'] = pd.to_datetime(sp500['DATE'])
# Output
sp500.head()

,TICKER,NAME,DATE,CIK,VARIABLE
0,MMM,3M Company,1976-08-09,0000066740,added_ticker
1,ABT,Abbott Laboratories,1964-03-31,0000001800,added_ticker
2,ABBV,AbbVie Inc.,2012-12-31,0001551152,added_ticker
3,ABMD,ABIOMED Inc,2018-05-31,0000815094,added_ticker
4,ACN,Accenture plc,2011-07-06,0001467373,added_ticker


In [4]:
sp500_adjustments = data[1]
sp500_adjustments = sp500_adjustments[2:].copy()
columns = ['date', 'added_ticker', 'added_name', 'removed_ticker', 'removed_name', 'reason']
sp500_adjustments.columns = columns
updates = sp500_adjustments[~sp500_adjustments['date'].str.contains(',')].T.shift(1).T
sp500_adjustments['date'].loc[~sp500_adjustments['date'].str.contains(',')] = np.nan
sp500_adjustments[sp500_adjustments['added_ticker'].isnull()]
sp500_adjustments.update(updates)
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T

sp500_additions = sp500_adjustments[~sp500_adjustments['added_ticker'].isnull()]
sp500_additions = sp500_additions[['date', 'added_ticker', 'added_name']]
sp500_additions.rename(columns={'added_name': 'name'}, inplace=True)
sp500_additions = pd.melt(sp500_additions, id_vars=['date','name'], value_vars=['added_ticker'])

sp500_deletions = sp500_adjustments[~sp500_adjustments['removed_ticker'].isnull()]
sp500_deletions = sp500_deletions[['date', 'removed_ticker', 'removed_name']]
sp500_deletions.rename(columns={'removed_name': 'name'}, inplace=True)
sp500_deletions = pd.melt(sp500_deletions, id_vars=['date','name'], value_vars=['removed_ticker'])

sp500_history = pd.concat([sp500_deletions, sp500_additions])
sp500_history.rename(columns = {'date':'DATE', 'name':'NAME', 'variable':'VARIABLE', 'value':'TICKER'}, inplace = True)
sp500_history['DATE'] = pd.to_datetime(sp500_history['DATE'])
sp500_history.head()

,DATE,NAME,VARIABLE,TICKER
0,2020-06-22,Nordstrom,removed_ticker,JWN
1,2020-05-22,Helmerich & Payne,removed_ticker,HP
2,2020-05-12,Capri Holdings,removed_ticker,CPRI
3,2020-05-12,Allergan,removed_ticker,AGN
4,2020-04-06,"Macy's, Inc.",removed_ticker,M


In [5]:
df = pd.concat([sp500, sp500_history], ignore_index=True)
df.sort_values(by='CIK', ascending=False, inplace=True)
deduped_df = df[~df.duplicated(['DATE', 'VARIABLE', 'TICKER'])].copy()
deduped_df.sort_values(by='DATE',inplace=True)
deduped_df = deduped_df[['TICKER', 'NAME', 'CIK', 'DATE', 'VARIABLE']]
deduped_df.to_csv("sp500_history.csv")
deduped_df.head()

C:\Users\nqabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,TICKER,NAME,CIK,DATE,VARIABLE
271,K,Kellogg Co.,0000055067,1957-01-01,added_ticker
296,L,Loews Corp.,0000060086,1957-01-01,added_ticker
473,VIAC,ViacomCBS,0001339947,1957-01-01,added_ticker
303,MAR,Marriott Int'l.,0001048286,1957-01-01,added_ticker
287,LEG,Leggett & Platt,0000058492,1957-01-01,added_ticker


In [6]:
deduped_df.sort_values(by='CIK', ascending=False, inplace=True)
deduped_df = deduped_df[~deduped_df.duplicated('TICKER')]
# discovery has 2 share classes listed
deduped_df = deduped_df[~deduped_df.duplicated('CIK')]
deduped_df.sort_values(by='TICKER', inplace=True)
deduped_df.drop(['DATE', 'VARIABLE'], axis=1, inplace=True)
deduped_df.to_csv("sp500_constituents.csv")
deduped_df.head()

,TICKER,NAME,CIK
11,A,Agilent Technologies Inc,0001090872
28,AAL,American Airlines Group,0000006201
8,AAP,Advance Auto Parts,0001158449
46,AAPL,Apple Inc.,0000320193
2,ABBV,AbbVie Inc.,0001551152
